# Import Libraries

In [1]:
import optuna
import os
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
import pandas as pd
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

In [2]:
def read_file(*args,file_type="csv")->pd.DataFrame:
    """:parameter input directories in sequence
        :return DataFrame"""
    
    path=os.path.join(*args)
    if file_type == "tsv":
        return pd.read_csv(path, sep="\t")
    return pd.read_csv(path)

# Experiments

In [3]:
mlflow.set_tracking_uri("http://127.0.0.1:5000/")

In [4]:
mlflow.set_experiment("DifferentModelsExperiments2")

<Experiment: artifact_location='/home/spynom/mlruns/14', creation_time=1729323387786, experiment_id='14', last_update_time=1729323387786, lifecycle_stage='active', name='DifferentModelsExperiments2', tags={}>

In [5]:
df=read_file("..","data","processed","cleaned.csv").dropna(how="any").drop_duplicates()

X=df.comment
y=df.category+1

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42,stratify=y)

ngram_range = (1, 3)  # Trigram setting
max_features = 10000  # Set max_features to 1000 for TF-IDF

# Vectorization using TF-IDF, fit on training data only
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
vectorizer.fit(X)  # Fit on all vocab data
X_train = vectorizer.transform(X_train)
X_test = vectorizer.transform(X_test)  # Transform test data

In [6]:
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        # Log model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_BalanceWeight_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Train model
        model.fit(X_train, y_train)
        train_y_pred=model.predict(X_train)
        test_y_pred = model.predict(X_test)

        # Log accuracy
        train_accuracy = accuracy_score(y_train,train_y_pred)
        test_accuracy = accuracy_score(y_test, test_y_pred)
        
        mlflow.log_metric("train_accuracy", train_accuracy)
        mlflow.log_metric("test_accuracy", test_accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, test_y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")

In [7]:
def objective_mnb(trial):
    alpha = trial.suggest_float('alpha', 1e-4, 1.0, log=True)  # Tuning the smoothing parameter

    # MultinomialNB model setup
    model = MultinomialNB(alpha=alpha)
    return accuracy_score(y_test, model.fit(X_train, y_train).predict(X_test))


# Step 7: Run Optuna for Multinomial Naive Bayes, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_mnb, n_trials=30)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    best_model = MultinomialNB(alpha=best_params['alpha'])

    # Log the best model with MLflow, passing the algo_name as "MultinomialNB"
    log_mlflow("MultinomialNB", best_model, X_train, X_test, y_train, y_test)

# Run the experiment for Multinomial Naive Bayes
run_optuna_experiment()

[I 2024-10-19 13:26:47,908] A new study created in memory with name: no-name-3e545b6e-cc0a-4a1b-8fbe-8b315abe1212
[I 2024-10-19 13:26:47,929] Trial 0 finished with value: 0.64969646799117 and parameters: {'alpha': 0.35797507249719124}. Best is trial 0 with value: 0.64969646799117.
[I 2024-10-19 13:26:47,946] Trial 1 finished with value: 0.641832229580574 and parameters: {'alpha': 0.5780388331428207}. Best is trial 0 with value: 0.64969646799117.
[I 2024-10-19 13:26:47,959] Trial 2 finished with value: 0.6568708609271523 and parameters: {'alpha': 0.0004325629154443956}. Best is trial 2 with value: 0.6568708609271523.
[I 2024-10-19 13:26:47,971] Trial 3 finished with value: 0.6564569536423841 and parameters: {'alpha': 0.0010396590066984262}. Best is trial 2 with value: 0.6568708609271523.
[I 2024-10-19 13:26:47,982] Trial 4 finished with value: 0.6567328918322296 and parameters: {'alpha': 0.00016136814614759368}. Best is trial 2 with value: 0.6568708609271523.
[I 2024-10-19 13:26:47,995]